# Using a Keras Convolutional Neural Network for classifying Flower Images

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from glob import glob
import cv2

## Importing images using OpenCV and Glob

In [ ]:
import os

In [ ]:
os.listdir("../input/flowers/flowers/")

In [ ]:
daisy_path = glob("../input/flowers/flowers/daisy/*jpg", recursive=True)
dandi_path = glob("../input/flowers/flowers/dandelion/*jpg", recursive=True)
rose_path = glob("../input/flowers/flowers/rose/*jpg", recursive=True)
sunflower_path = glob("../input/flowers/flowers/sunflower/*jpg", recursive=True)
tulip_path = glob("../input/flowers/flowers/tulip/*jpg", recursive=True)

1. Above, we have defined the paths to the flowers dataset images.
2. In the below step, we read image files into the dataset using OpenCV (`cv2`)

In [ ]:
daisy_images = [cv2.imread(file) for file in daisy_path]
dandi_images = [cv2.imread(file) for file in dandi_path]
rose_images = [cv2.imread(file) for file in rose_path]
sunflower_images = [cv2.imread(file) for file in sunflower_path]
tulip_images = [cv2.imread(file) for file in tulip_path]

Print 4 random images from the daisy_images set

In [ ]:
print("Shapes of different daisy images and what they look like\n")
def print_image(img):
    plt.figure()
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(img.shape)
    plt.show()

for num in np.random.choice(daisy_images, size = 4):
    print_image(num)

In [ ]:
def resize_image(image):
    return cv2.resize(image, (320,320))

In [ ]:
reshaped_daisy = [resize_image(img) for img in daisy_images]
reshaped_dandi = [resize_image(img) for img in dandi_images]
reshaped_rose = [resize_image(img) for img in rose_images]
reshaped_sunflower = [resize_image(img) for img in sunflower_images]
reshaped_tulip = [resize_image(img) for img in tulip_images]

In [ ]:
num_daisy, num_dandi, num_rose, num_sunflower, num_tulip = len(reshaped_daisy), len(reshaped_dandi), len(reshaped_rose), len(reshaped_sunflower), len(reshaped_tulip)

In [ ]:
labels_daisy = ['daisy']*num_daisy
labels_dandi = ['dandi']*num_dandi
labels_rose = ['rose']*num_rose
labels_sunflower = ['sunflower']*num_sunflower
labels_tulip = ['tulip']*num_tulip

In [ ]:
labels = np.concatenate((labels_daisy, labels_dandi, labels_rose, labels_sunflower, labels_tulip))

In [ ]:
images = np.concatenate((reshaped_daisy, reshaped_dandi, reshaped_rose, reshaped_sunflower, reshaped_tulip))

In [ ]:
print("Images data shape:", images.shape)
print("Labels data shape:", labels.shape)

In [ ]:
n_classes = len(np.unique(labels))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, BatchNormalization, Flatten
from keras.regularizers import l1_l2
from keras.optimizers import Adam

In [ ]:
#Params

n_hidden = 50
l1, l2 = 0.15, 0.15
dropoutpc = 0.25

In [ ]:
flw_model = Sequential([
    Conv2D(filters=10,kernel_size=(2,2), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same', input_shape = (320,320,3), data_format='channels_last'),
    MaxPool2D((2,2)),
    Conv2D(filters=20,kernel_size=(2,2), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same'),
    Conv2D(filters=20,kernel_size=(4,4), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same'),
    MaxPool2D((3,3)),
    Conv2D(filters=30,kernel_size=(2,2), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same'),
    Conv2D(filters=30,kernel_size=(4,4), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same'),
    MaxPool2D((2,2)),
    Conv2D(filters=40,kernel_size=(2,2), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same'),
    Conv2D(filters=40,kernel_size=(4,4), activation='relu', kernel_regularizer=l1_l2(l1, l2), strides=1, padding='same'),
    MaxPool2D((2,2)),
    Dense(n_hidden),
    Flatten(),
    BatchNormalization(),
    Dropout(dropoutpc),
    Dense(n_classes, activation="softmax")    
])

In [ ]:
flw_model.summary()

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
label_dummies = pd.get_dummies(labels).as_matrix()
print(label_dummies.shape)

In [ ]:
trainX, testX, trainY_cat, testY_cat = train_test_split(images, labels, test_size = 0.2)
trainY = pd.get_dummies(trainY_cat).as_matrix()
testY = pd.get_dummies(testY_cat).as_matrix()


In [ ]:
batchsize = 16
epochs = 32
lr = 1e-4
decay = 0.1 * lr

opt = Adam(lr, decay)

In [ ]:
flw_model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['acc'])

In [ ]:
history = flw_model.fit(x = trainX, y= trainY, batch_size = batchsize, epochs = epochs, validation_split= 0.2, verbose=1)

In [ ]:
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(figsize = (16,4), title = "Loss")
pd.DataFrame(history.history)[['acc', 'val_acc']].plot(figsize = (16,4), title = "Accuracy")

In [ ]:
test_pred = flw_model.predict_classes(testX).reshape((len(test_pred),1))
print(test_pred.shape)

testY_labels = np.reshape(np.argmax(testY, axis=1), (-1,1))

precision_score(testY_labels, test_pred, average='macro'), recall_score(testY_labels, test_pred, average='macro'), f1_score(testY_labels, test_pred, average='macro')